# Compute scores

In [ ]:
import pandas as pd

from config import QUANTILES, ROOT
from src.load_data import load_nowcasts, load_predictions
from src.scoring_functions import evaluate_models

In [ ]:
PATH_SCORES = ROOT / "results" / "scores"
PATH_SCORES.mkdir(parents=True, exist_ok=True)

In [ ]:
df = load_predictions()
df_nowcasts = load_nowcasts(quantiles=QUANTILES)
df = pd.concat([df, df_nowcasts], ignore_index=True)

In [ ]:
df.model.unique()

# Aggregated scores

In [ ]:
df_national = evaluate_models(df, "national")
df_age = evaluate_models(df, "age")

scores = pd.concat(
    [df_national.assign(level="national"), df_age.assign(level="age")],
    ignore_index=True,
)

scores["level"] = pd.Categorical(scores["level"], categories=["national", "age"], ordered=True)

In [ ]:
scores.to_csv(PATH_SCORES / "scores.csv", float_format="%.3f", index=False)

## By age group

In [ ]:
scores_age = evaluate_models(df, "age", by_age=True)

In [ ]:
scores_age.to_csv(PATH_SCORES / "scores_age.csv", float_format="%.3f", index=False)

## By horizon

In [ ]:
df_national = evaluate_models(df, "national", by_horizon=True)
df_age = evaluate_models(df, "age", by_horizon=True)

scores_horizon = pd.concat(
    [df_national.assign(level="national"), df_age.assign(level="age")],
    ignore_index=True,
)

In [ ]:
scores_horizon.to_csv(PATH_SCORES / "scores_horizon.csv", float_format="%.3f", index=False)